In [1]:
import modelos as md
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys

In [2]:
import urllib.request

print('Beginning file download with urllib2...')

url = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'
urllib.request.urlretrieve(url, '/Users/aurelianosancho/Sua equipe Dropbox/Aureliano Sancho/COVID-19/Modelos/estados_bruto.csv')

Beginning file download with urllib2...


('/Users/aurelianosancho/Sua equipe Dropbox/Aureliano Sancho/COVID-19/Modelos/estados_bruto.csv',
 <http.client.HTTPMessage at 0x1521da1790>)

In [3]:
data = [[11, 'Rondônia', 'RO'], [12, 'Acre', 'AC'], [13, 'Amazonas', 'AM'], [14 , 'Roraima', 'RO'], 
        [15 , 'Pará', 'PA'], [16 , 'Amapá', 'AP'], [17 , 'Tocantins', 'TO'], 
        [21 , 'Maranhão', 'MA'], [22, 'Piauí', 'PI'], [23 , 'Ceará', 'CE'], [24 , 'Rio Grande do Norte', 'RN'], 
        [25 , 'Paraíba', 'PB'], [26 , 'Pernambuco', 'PE'], [27 , 'Alagoas', 'AL'], [28 , 'Sergipe', 'SE'], 
        [29 , 'Bahia', 'BA'], 
        [31 , 'Minas Gerais', 'MG'], [32 , 'Espírito Santo', 'ES'], [33 , 'Rio de Janeiro', 'RJ'], 
        [34 , 'São Paulo', 'SP'],
        [41, 'Paraná', 'PR'], [42, 'Santa Catarina', 'SC'], [43, 'Rio Grande do Sul', 'RS'], 
        [50, 'Mato Grosso do Sul', 'MS'], [51, 'Mato Grosso', 'MT'], [52, 'Goiás', 'GO'],
        [53, 'Distrito Federal', 'DF']] 
  
# Create the pandas DataFrame 
df0 = pd.DataFrame(data, columns = ['UF', 'Nomes', 'sigla']) 
#df0 = df0.drop(['state'], axis=1)

df1 = pd.read_csv('estados_bruto.csv')
df1 = df1.drop(['country', 'city'], axis=1)

df3 = df1.merge(df0, left_on='state', right_on='sigla', suffixes=(False, False))
df3 = df3.drop(['sigla','state'], axis=1)
df3.columns = ['date', 'deaths', 'newcases', 'totalcases', 'UF', 'state']
df3.to_csv('estadosnet.csv', encoding='utf-8', index=False)

##########################

In [5]:
# -*- coding: utf-8 -*-
"""
Created on Sat Mar 21 13:22:11 2020

@author: Rafael Veiga
"""

# parametros
modelo_usado = 'SIR_EDO' # SIR, SIR_EDO , SEIR_EDO ou SEQIJR_EDO
N_inicial = 1000
min_cases = 10
min_dias = 10

arq_saida = '/Users/aurelianosancho/Sua equipe Dropbox/Aureliano Sancho/COVID-19/Modelos/estados.csv'
arq_sumario = '/Users/aurelianosancho/Sua equipe Dropbox/Aureliano Sancho/COVID-19/Modelos/estado_sumario.csv'
arq_brasil_saida = '/Users/aurelianosancho/Sua equipe Dropbox/Aureliano Sancho/COVID-19/Modelos/brasil.csv'
previsao_ate = dt.date(2020,4,5)

#carregar dados
nome, local = md.ler_banco('/Users/aurelianosancho/Sua equipe Dropbox/Aureliano Sancho/COVID-19/Modelos/estadosnet.csv','state')

novo_nome = []
nome.insert(0,'Brasil')
novo_local = []
#brasil
br = local[0]
for i in range(1,len(local)):
    br = br.append(local[i],ignore_index=True)
x = list(br.date.unique())
x = sorted(x)
y = []
for d in x:
    y.append(sum(br[br.date==d].totalcasos))

newy = [y[0]]
for i in range(1,len(y)):
    newy.append(y[i]-y[i-1])
state = ['Brasil' for i in y]
UF = [np.nan for i in y]
brasil = pd.DataFrame({'date':x,'state':state,'UF':UF,'novosCasos':newy,'totalcasos':y})
local.insert(0,brasil)
for i in range(len(nome)):
    if (local[i].totalcasos.iloc[-1]>=min_cases) & (len(local[i])>=10):
        novo_local.append(local[i])
        novo_nome.append(nome[i])
previsao_ate = previsao_ate + dt.timedelta(1)
modelos = []
for i in range(len(novo_nome)):
    print("\n\n"+str(nome[i])+'\n')
    modelo = None
    if modelo_usado =='SIR':
        modelo = md.SIR(N_inicial)
    elif modelo_usado =='SIR_EDO':
        modelo = md.SIR_EDO(N_inicial)
    elif modelo_usado =='SEIR_EDO':
        modelo = md.SEIR_EDO(N_inicial)
    elif modelo_usado=='SEQIJR_EDO':
        modelo = md.SEQIJR_EDO(N_inicial)
    else:
        print('Modelo desconhecido '+modelo_usado)
        sys.exit(1)
    
    y = novo_local[i].totalcasos
    x = range(1,len(y)+1)
    modelo.fit(x,y,name=novo_nome[i])
    modelos.append(modelo)
    dias = (previsao_ate-novo_local[i].date.iloc[0]).days
    x_pred = range(1,dias+1)
    y_pred =modelo.predict(x_pred)
#    plt.plot(y_pred,c='r',label='Predição Infectados')
#    plt.plot(y,c='b',label='Infectados')
#    plt.legend(fontsize=15)
#    plt.title('Dinâmica do CoviD19 - {}'.format(nome[i]),fontsize=20)
#    plt.ylabel('Casos COnfirmados',fontsize=15)
#    plt.xlabel('Dias',fontsize=15)
#    plt.show()    
    novo_local[i]['casos_preditos'] = y_pred[0:len(novo_local[i])]
    ultimo_dia = novo_local[i].date.iloc[-1]
    dias = (previsao_ate-novo_local[i].date.iloc[-1]).days
    for d in range(1,dias):
        di = d+len(x)-1
        novo_local[i]=novo_local[i].append({'casos_preditos':y_pred[di],'date':ultimo_dia+dt.timedelta(d),'state':novo_local[i].state.iloc[0]}, ignore_index=True)
    
brasil =   novo_local[0]
del brasil['UF']  
brasil.to_csv(arq_brasil_saida,index=False)    
df = novo_local[1]
for i in range(2,len(novo_local)):
    df = df.append(novo_local[i],ignore_index=True)
df.to_csv(arq_saida,index=False)

su = pd.DataFrame()
su['state'] = novo_nome

coef_list = []
y = []
coef_name = None
for i in range(len(novo_nome)):
    y.append(';'.join(map(str, modelos[i].y)))
    coef_name, coef  = modelos[i].getCoef()
    coef_list.append(coef)

for c in range(len(coef_name)-1):
    l = []
    for i in range(len(coef_list)):
        l.append(coef_list[i][c])
    su[coef_name[c]]=l
coef_name = coef_name[-1]
for c in range(len(coef_name)):
    l=[]
    for i in range(len(coef_list)):
       l.append(coef_list[i][-1][c]) 
    su[coef_name[c]] = l

su['y'] = y
su.to_csv(arq_sumario,index=False)

#%matplotlib qt
#for i in range(len(local[i])):
#    modelos[i].plot(nome[i])

/Users/aurelianosancho/Sua equipe Dropbox/Aureliano Sancho/COVID-19/Modelos/modelos.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  banco.date[i] = dt.datetime.strptime(banco.date[i], '%Y-%m-%d').date()
2020-03-28 21:22:53,135 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.
2020-03-28 21:22:53,145 - Platypus - INFO - NSGAII starting




Brasil



2020-03-28 21:22:55,146 - Platypus - INFO - NSGAII finished; Total NFE: 1048, Elapsed Time: 0:00:02.001314
2020-03-28 21:22:55,193 - Platypus - INFO - NSGAII starting




São Paulo



2020-03-28 21:22:57,023 - Platypus - INFO - NSGAII finished; Total NFE: 1044, Elapsed Time: 0:00:01.829783
2020-03-28 21:22:57,124 - Platypus - INFO - NSGAII starting


...

2020-03-28 21:23:36,054 - Platypus - INFO - NSGAII finished; Total NFE: 1046, Elapsed Time: 0:00:01.514523
2020-03-28 21:23:36,114 - Platypus - INFO - NSGAII starting




Pará



2020-03-28 21:23:37,313 - Platypus - INFO - NSGAII finished; Total NFE: 1043, Elapsed Time: 0:00:01.198742
2020-03-28 21:23:37,366 - Platypus - INFO - NSGAII starting




Paraíba



2020-03-28 21:23:38,658 - Platypus - INFO - NSGAII finished; Total NFE: 1038, Elapsed Time: 0:00:01.292097


In [6]:
pd.set_option('display.max_rows', None)
df = pd.read_csv('estado_sumario.csv')
df.head()

,state,beta,gamma,R0,S,I,R,y
0,Brasil,0.392798,0.073684,0.187587,-0.001553,-0.015006,0.016559,1;1;1;1;2;2;2;2;3;8;13;19;25;30;35;69;109;120;...
1,São Paulo,0.368825,0.072215,0.195798,-0.002626,-0.016517,0.019143,1;1;1;1;2;2;2;2;3;6;10;13;16;16;19;45;64;64;65...
2,Rio de Janeiro,0.416375,0.082054,0.197068,-0.006803,-0.021894,0.028698,1;1;2;3;8;8;14;16;19;24;24;31;39;63;66;109;119...
3,Espírito Santo,0.394280,0.200000,0.507254,-0.031851,0.002370,0.029480,1;1;1;1;1;1;1;2;2;4;4;8;8;11;13;16;26;26;33;40...
4,Bahia,0.454365,0.200000,0.440175,-0.023202,-0.010870,0.034072,1;2;2;2;2;3;3;7;7;9;10;16;27;31;34;41;55;63;79...


In [7]:
ll = len(df)

22

In [8]:
%matplotlib qt
for i in range(ll):
    if (len(local[i]) > 14):
        plt.figure(i)
        modelos[i].plot(nome[i])
        plt.show()

In [9]:
#modelos[16].plot(nome[16])              ### 
#modelos[2].plot(nome[2])
#modelos[4].plot(nome[4])